In [ ]:
# ✅ STEP 1: UPLOAD MOVIELENS FILES
from google.colab import files
import pandas as pd

print("📂 Please upload movies.csv and ratings.csv")
uploaded = files.upload()

# ✅ STEP 2: LOAD THE DATASETS
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

print("✅ Movies loaded:", movies.shape)
print("✅ Ratings loaded:", ratings.shape)

# ✅ STEP 3: BUILD MODELS
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# --- Collaborative Filtering ---
movie_data = pd.merge(ratings, movies, on='movieId')
user_movie_matrix = movie_data.pivot_table(index='userId', columns='title', values='rating').fillna(0)
collab_similarity = cosine_similarity(user_movie_matrix.T)
title_to_index = pd.Series(data=range(len(user_movie_matrix.columns)), index=user_movie_matrix.columns)

def recommend_collaborative(movie_title, top_n=5):
    """ Recommend movies using collaborative filtering """
    if movie_title not in title_to_index:
        return ["❌ Movie not found in dataset."]
    idx = title_to_index[movie_title]
    sim_scores = list(enumerate(collab_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    return [user_movie_matrix.columns[i] for i, _ in sim_scores]

# --- Content-Based Filtering (Genres) ---
movies['genres'] = movies['genres'].fillna('').replace("(no genres listed)", "")
tfidf = TfidfVectorizer(stop_words='english')
genre_matrix = tfidf.fit_transform(movies['genres'])
genre_similarity = cosine_similarity(genre_matrix)

def recommend_by_genre(movie_title, top_n=5):
    """ Recommend movies using genres """
    if movie_title not in movies['title'].values:
        return ["❌ Movie not found in dataset."]
    idx = movies[movies['title'] == movie_title].index[0]
    sim_scores = list(enumerate(genre_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    return movies['title'].iloc[[i for i, _ in sim_scores]].tolist()

# ✅ STEP 4: USER INPUT & RECOMMENDATIONS
movie_name = input("🎬 Enter a movie you like (exact title, e.g., Toy Story (1995)): ")

print("\n--- 🎯 Top 5 Collaborative Filtering Recommendations ---")
for i, movie in enumerate(recommend_collaborative(movie_name), 1):
    print(f"{i}. {movie}")

print("\n--- 🎯 Top 5 Content-Based (Genre) Recommendations ---")
for i, movie in enumerate(recommend_by_genre(movie_name), 1):
    print(f"{i}. {movie}")


📂 Please upload movies.csv and ratings.csv


Saving ml-latest-small (2).zip to ml-latest-small (2) (4).zip
✅ Movies loaded: (10, 3)
✅ Ratings loaded: (10, 4)
